# Utah Real Estate Market Analysis: Sales Trends and Predictive Modeling

## Overview

The Utah Real Estate Sales Analysis project aims to leverage a dataset containing detailed real estate listings from Utah to explore market trends, pricing dynamics, and key property characteristics influencing sales prices. The dataset provides comprehensive information about 4,440 unique real estate listings, including attributes such as property type, size, listing price, bedrooms, bathrooms, lot size, garage spaces, and the date of last sale. By analyzing this data, we seek to gain insights into the Utah real estate market and develop predictive models to estimate property prices based on various features.

# Exploring the Data

## Data Understanding
As previously mentioned, the dataset used in this analysis is sourced from Utah real estate listings and provides detailed information for 4,440 properties, which were built as early as 1860 to 2026 for future listings. The dataset allows for a comprehensive analysis of factors influencing property values and trends within Utah’s real estate market.

## 1.1 Load the Datasets
During this process, we will load and prepare our data for an exploratory data analysis by gaining key insights from our data by taking a general look (.head), checking for missing values (.info), and a brief summary of statistics (.describe).

In [1]:
#Import and load the data with panda
import pandas as pd
path = '../data/real_estate_utah.csv'
real_estate_data = pd.read_csv(path)

In [ ]:
real_estate_data.head()

In [ ]:
real_estate_data.info()

In [ ]:
real_estate_data.describe()

## 1.2 Understanding the Structure

### Real Estate Data Key Features
Column Descriptors (Pulled from Kaggle)
- type: Type of property (e.g., single_family, land)
- text: Description of the property
- year_built: Year the property was built
- beds: Number of bedrooms
- baths: Total number of bathrooms
- baths_full: Number of full bathrooms
- baths_half: Number of half bathrooms
- garage: Number of garage spaces
- lot_sqft: Lot size in square feet
- sqft: Property size in square feet
- stories: Number of stories
- lastSoldOn: Date the property was last sold
- listPrice: Listing price of the property
- status: Current status of the property (e.g., for_sale)

Key Data Insights based on Summary Stats from .describe:
- potential outliers in the sqft, lot_sqft, and listPrice columns
- Data types are generally correct except for lastSoldOn, type, and status

# Data Cleaning

Initial exploration reveals possible outliers in variables like lot_sqft, sqft, and listPrice, with some unusually high values that may skew analysis. Handling these outliers will involve filtering or capping based on the overall distribution. The lastSoldOn column, representing dates, will be transformed into a more analytical format (e.g., extracting year or month). For categorical variables like type and status, label encoding or one-hot encoding will be applied based on the context of each analysis. After completing these transformations, the cleaned dataset will be exported in a standardized format for seamless integration with other analysis notebooks.

## 2.1 Handle Missing Values
Our .info does not show any current missing null values, but for a sanity check let's take a look for null values again in case we missed anything using isnull() and sum() to identify columns with missing data.


In [ ]:
real_estate_data.isnull().sum()

## 2.2 Detect and Removing Outliers
Potential outliers with abnormally high values that might affect the study were found in the lot_sqft, sqft, and listPrice columns during preliminary investigation. Depending on the distribution of the data, we will identify these outliers and implement filtering or capping using numpy

In [6]:
import numpy as np

# Set thresholds based on percentiles to cap or filter out extreme values
cap_thresholds = {
    'lot_sqft': np.percentile(real_estate_data['lot_sqft'], 99),
    'sqft': np.percentile(real_estate_data['sqft'], 99),
    'listPrice': np.percentile(real_estate_data['listPrice'], 99)
}

# Capping outliers at the 99th percentile
real_estate_data['lot_sqft'] = np.where(real_estate_data['lot_sqft'] > cap_thresholds['lot_sqft'], cap_thresholds['lot_sqft'], real_estate_data['lot_sqft'])
real_estate_data['sqft'] = np.where(real_estate_data['sqft'] > cap_thresholds['sqft'], cap_thresholds['sqft'], real_estate_data['sqft'])
real_estate_data['listPrice'] = np.where(real_estate_data['listPrice'] > cap_thresholds['listPrice'], cap_thresholds['listPrice'], real_estate_data['listPrice'])


In [ ]:
# Check summary statistics again after capping to see if extreme values remain
real_estate_data[['lot_sqft', 'sqft', 'listPrice']].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])


After handling the outliers, it appears that extreme values in "lot_sqft," "sqft," and "listPrice" have been capped. The maximum values have been significantly reduced, especially in the "lot_sqft" and "listPrice" columns, where the original maximums were in the range of tens of millions, but the capped values are much lower, under 7 million for "listPrice" and around 6 million for "lot_sqft." The 99th percentile also shows more reasonable values after capping.

## 2.3 Date Standardization for lastSoldOn Column
The lastSoldOn column, representing dates, will be converted to DateTime format, allowing extraction of specific components like year and month for deeper time-based analysis.

In [8]:
# Convert 'lastSoldOn' to datetime
real_estate_data['lastSoldOn'] = pd.to_datetime(real_estate_data['lastSoldOn'], errors='coerce')

# Extract year and month for analysis
real_estate_data['sold_year'] = real_estate_data['lastSoldOn'].dt.year
real_estate_data['sold_month'] = real_estate_data['lastSoldOn'].dt.month


In [ ]:
#Check for new columns
real_estate_data.head()

## 2.4 Encoding Categorical Variables
Categorical variables like type and status will be encoded. For variables with ordinal significance, label encoding will be used, while one-hot encoding will apply for nominal categories.

### Handling the Type Column:
We are going to create a new column that categorizes the 'type' column into individual property types compared to "Other". For example, if the property type is "Single Family", it will remain as "Single Family", but if it's any other type, it will be classified as "Other". We are doing this to keep the original type column to answer questions for EDA, but drop it once we get to constructing the machine learning model. Perform one-hot encoding on the "type" column to create separate columns for each individual type. Concatenate both the new comparison column and the one-hot encoded columns.

In [5]:
# Create a new 'type_comparison' column where each 'type' is compared to 'Other'
real_estate_data['type_comparison'] = real_estate_data['type'].apply(lambda x: x if x == 'Single Family' else 'Other')

# One-hot encoding for the 'type' column (original categorical column)
type_dummies = pd.get_dummies(real_estate_data['type'], prefix='type')

# Concatenate the new column and the one-hot encoded columns with the original DataFrame
real_estate_data = pd.concat([real_estate_data, type_dummies], axis=1)


### Handling the Status Column

For the label encoding the status: 0 represents if the property is for sale and 1 represents if the property is ready to build

In [6]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding for ordinal categories (status)
label_encoder = LabelEncoder()
real_estate_data['status_encoded'] = label_encoder.fit_transform(real_estate_data['status'])
real_estate_data['status_encoded'].value_counts()

status_encoded
0    4185
1     255
Name: count, dtype: int64

## 2.5 Exporting the Cleaned Dataset
After completing these transformations, the cleaned dataset will be exported in a standardized format to integrate seamlessly with other analysis notebooks.